# SID510460295_AssgnOpt2 Intelligent Animation

### Imports

In [5]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
import time
import sys
import random
import json

### Paths

In [54]:
path_to_original_mov = "Opt1-MarionetteMovements.mov"
frame_save_path = "./movie_frames/"
body_data_file = "./body_data.json"

### Capture movie frames

In [3]:
cap = cv2.VideoCapture(path_to_original_mov)
if not cap.isOpened():
    print('{} not opened'.format(path_to_original_mov))
    sys.exit(1)
time_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_counter = 0                                             # FRAME_COUNTER
while(1):
    return_flag, frame = cap.read()  
    if not return_flag:
        print('Video Reach End')
        break
    # Main Content - Start
    cv2.imshow('Monkey', frame)
    cv2.imwrite(frame_save_path + 'frame%d.tif' % frame_counter, frame)
    frame_counter += 1
    # Main Content - End    
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Video Reach End


### Globals

In [86]:
FRAME_RATE = 30
FRAME_WIDTH = int(frame_width)
FRAME_HEIGHT = int(frame_height)
TOTAL_FRAMES = time_length


CENTERS_MAX_DISTANCE = 25
THRESHOLD = 300

### Create data structure for storing movements, and a way to add to that data structure

In [106]:
# Create json function for storing limb location

data = {
    "frames" : [
    #     {
    #         "number": 0,
    #         "chest" : {
    #             "x" : 0,
    #             "y" : 0
    #         },
    #         "left_hand" : {
    #             "x" : 0,
    #             "y" : 0
    #         },
    #         "right_hand" : {
    #             "x" : 0,
    #             "y" : 0
    #         },
    #         "left_foot" : {
    #             "x" : 0,
    #             "y" : 0
    #         },
    #         "right_foot" : {
    #             "x" : 0,
    #             "y" : 0
    #         }
    #     }
    ]
}

def add_frame_to_json(frame_number, chest_x, chest_y, left_hand_x, left_hand_y, right_hand_x, right_hand_y, left_foot_x, left_foot_y, right_foot_x, right_foot_y):
    # Check if this is the first frame
    if not data["frames"]:
        # If it is, add the frame with the given values
        data["frames"].append({
            "number": frame_number,
            "chest" : {
                "x" : chest_x,
                "y" : chest_y
            },
            "left_hand" : {
                "x" : left_hand_x,
                "y" : left_hand_y
            },
            "right_hand" : {
                "x" : right_hand_x,
                "y" : right_hand_y
            },
            "left_foot" : {
                "x" : left_foot_x,
                "y" : left_foot_y
            },
            "right_foot" : {
                "x" : right_foot_x,
                "y" : right_foot_y
            }
        })
    else:
        # If it's not the first frame, get the previous frame's values
        prev_frame = data["frames"][-1]
        prev_chest_x = prev_frame["chest"]["x"]
        prev_chest_y = prev_frame["chest"]["y"]
        prev_left_hand_x = prev_frame["left_hand"]["x"]
        prev_left_hand_y = prev_frame["left_hand"]["y"]
        prev_right_hand_x = prev_frame["right_hand"]["x"]
        prev_right_hand_y = prev_frame["right_hand"]["y"]
        prev_left_foot_x = prev_frame["left_foot"]["x"]
        prev_left_foot_y = prev_frame["left_foot"]["y"]
        prev_right_foot_x = prev_frame["right_foot"]["x"]
        prev_right_foot_y = prev_frame["right_foot"]["y"]
        
        # Use the previous frame's values if a None value is passed in
        chest_x = chest_x if chest_x is not None else prev_chest_x
        chest_y = chest_y if chest_y is not None else prev_chest_y
        left_hand_x = left_hand_x if left_hand_x is not None else prev_left_hand_x
        left_hand_y = left_hand_y if left_hand_y is not None else prev_left_hand_y
        right_hand_x = right_hand_x if right_hand_x is not None else prev_right_hand_x
        right_hand_y = right_hand_y if right_hand_y is not None else prev_right_hand_y
        left_foot_x = left_foot_x if left_foot_x is not None else prev_left_foot_x
        left_foot_y = left_foot_y if left_foot_y is not None else prev_left_foot_y
        right_foot_x = right_foot_x if right_foot_x is not None else prev_right_foot_x
        right_foot_y = right_foot_y if right_foot_y is not None else prev_right_foot_y
        
        # Add the frame with the updated values
        data["frames"].append({
            "number": frame_number,
            "chest" : {
                "x" : chest_x,
                "y" : chest_y
            },
            "left_hand" : {
                "x" : left_hand_x,
                "y" : left_hand_y
            },
            "right_hand" : {
                "x" : right_hand_x,
                "y" : right_hand_y
            },
            "left_foot" : {
                "x" : left_foot_x,
                "y" : left_foot_y
            },
            "right_foot" : {
                "x" : right_foot_x,
                "y" : right_foot_y
            }
        })

def get_body_data():
    with open(body_data_file) as json_file:
        return json.load(json_file)

def write_body_data():
    with open(body_data_file, 'w') as outfile:
        json.dump(data, outfile)

### Create functionality to identify where the monkeys red spots are

In [44]:
def convert_frame_to_black_and_white(frame):
    # Make it so that the frame is all black except for the colour red which is white
    # This will make it easier to find the limbs
    # Convert the frame to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
    # Define the range of red colour in HSV
    lower_red = np.array([0,150,150])
    upper_red = np.array([10,255,255])
    # Threshold the HSV image to get only red colours
    mask = cv2.inRange(hsv, lower_red, upper_red)
    # Bitwise-AND mask and original image
    frame = cv2.bitwise_and(frame,frame, mask= mask)
    # Convert the frame to grayscale
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Convert the frame to black and white
    ret, frame = cv2.threshold(frame, 0, 255, cv2.THRESH_BINARY)
    return frame

def find_patch_centers(img):
    # Find contours in the image
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Calculate the centroid of each contour
    centers = []
    for contour in contours:
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            centers.append((cx, cy))
    
    return centers

def combine_centers(centers, max_distance):
    # Create a copy of the centers list
    new_centers = centers.copy()
    
    # Loop through each center
    i = 0
    while i < len(new_centers):
        # Loop through each center again starting from the next one
        j = i + 1
        while j < len(new_centers):
            # Calculate the distance between the two centers
            dist = np.sqrt((new_centers[i][0] - new_centers[j][0])**2 + (new_centers[i][1] - new_centers[j][1])**2)
            # If the distance is less than max_distance, combine the centers
            if dist < max_distance:
                new_centers[i] = ((new_centers[i][0] + new_centers[j][0]) // 2, (new_centers[i][1] + new_centers[j][1]) // 2)
                del new_centers[j]
            else:
                j += 1
        i += 1
    
    return new_centers

### Generate all frames with red dots, and convert to a movie

This is to verify that the CENTERS_MAX_DISTANCE chosen is reasonable.

In [108]:
# Open the video file for writing
path_to_output_video = './monkeyWithCenters.mp4'
out = cv2.VideoWriter(path_to_output_video, cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 30, (int(frame_width), int(frame_height)))

all_centers = []

# Loop through each frame and write it to the video file
for frame_number in range(0, TOTAL_FRAMES):
    # Load the frame
    img = cv2.imread('movie_frames/frame%d.tif' % frame_number)
    # Swap the blue and red channels
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Convert the frame to black and white
    bw_img = convert_frame_to_black_and_white(img)
    # Find the centers of the patches
    centers = find_patch_centers(bw_img)
    # Combine the centers that are too close together
    centers = combine_centers(centers, CENTERS_MAX_DISTANCE)
    # Create frame with red dots over top of it and write it to the video file
    for center in centers:
        img = cv2.circle(img, center, radius=3, color=(255, 0, 0), thickness=-1)
    all_centers.append(centers)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite('movie_frames_with_centers/frame%d.tif' % frame_number, img)
    out.write(img)

# Release the video file
out.release()
cv2.destroyAllWindows()

### Create JSON data structure
After examining the movie produced, I believe the centers look like they are in relatively good positions. Now it is time to extract those centers into the JSON data format

In [95]:
# def distance(point1, point2):
#     return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# # Assign the initial limb positions
# first_frame_centers = all_centers[0]

# # Sort the list in order of the y values
# first_frame_centers.sort(key=lambda tup: tup[1])

# initial_chest_position = first_frame_centers[len(first_frame_centers) // 2]

# initial_hand_positions = first_frame_centers[:len(first_frame_centers) // 2]

# initial_hand_positions.sort(key=lambda tup: tup[0])

# initial_left_hand_position = initial_hand_positions[0]
# initial_right_hand_position = initial_hand_positions[1]

# initial_foot_positions = first_frame_centers[len(first_frame_centers) // 2 + 1:]

# initial_foot_positions.sort(key=lambda tup: tup[0])

# initial_left_foot_position = initial_foot_positions[0]
# initial_right_foot_position = initial_foot_positions[1]

# # Add the initial frame to the json
# add_frame_to_json(0, initial_chest_position[0], initial_chest_position[1], initial_left_hand_position[0], initial_left_hand_position[1], initial_right_hand_position[0], initial_right_hand_position[1], initial_left_foot_position[0], initial_left_foot_position[1], initial_right_foot_position[0], initial_right_foot_position[1])

# # Loop through each frame and find the limbs
# prev_chest_pos = initial_chest_position
# prev_left_hand_pos = initial_left_hand_position
# prev_right_hand_pos = initial_right_hand_position
# prev_left_foot_pos = initial_left_foot_position
# prev_right_foot_pos = initial_right_foot_position

# for frame_number in range(1, TOTAL_FRAMES):
#     current_frame_centers = all_centers[frame_number]
#     current_chest_pos = None
#     current_left_hand_pos = None
#     current_right_hand_pos = None
#     current_left_foot_pos = None
#     current_right_foot_pos = None

#     # Assign each limb to the center that is closest to it according to the previous frame
#     for center in current_frame_centers:
#         distances = {
#             "chest": distance(center, prev_chest_pos),
#             "left_hand": distance(center, prev_left_hand_pos),
#             "right_hand": distance(center, prev_right_hand_pos),
#             "left_foot": distance(center, prev_left_foot_pos),
#             "right_foot": distance(center, prev_right_foot_pos)
#         }
#         closest_limb = min(distances, key=distances.get)
#         if closest_limb == "chest":
#             current_chest_pos = center
#         elif closest_limb == "left_hand":
#             current_left_hand_pos = center
#         elif closest_limb == "right_hand":
#             current_right_hand_pos = center
#         elif closest_limb == "left_foot":
#             current_left_foot_pos = center
#         elif closest_limb == "right_foot":
#             current_right_foot_pos = center

#     # If two limbs are allocated to the same center, find the one that is closest and allocate that limb to the center, and the other to the center that is next closest to it
#     while True:
#         allocated_centers = [current_chest_pos, current_left_hand_pos, current_right_hand_pos, current_left_foot_pos, current_right_foot_pos]
#         allocated_centers = [center for center in allocated_centers if center is not None]
#         unique_centers = list(set(allocated_centers))
#         if len(unique_centers) == len(allocated_centers):
#             break
#         for center in unique_centers:
#             limbs_at_center = [limb for limb in allocated_centers if limb == center]
#             if len(limbs_at_center) > 1:
#                 distances = {}
#                 for limb in limbs_at_center:
#                     for other_center in unique_centers:
#                         if other_center != center:
#                             distances[(limb, other_center)] = distance(limb, other_center)
#                 closest_limb, closest_center = min(distances, key=distances.get)
#                 if closest_limb == current_chest_pos:
#                     current_chest_pos = None
#                     if closest_center == current_left_hand_pos:
#                         current_left_hand_pos = center
#                     elif closest_center == current_right_hand_pos:
#                         current_right_hand_pos = center
#                     elif closest_center == current_left_foot_pos:
#                         current_left_foot_pos = center
#                     elif closest_center == current_right_foot_pos:
#                         current_right_foot_pos = center
#                 elif closest_limb == current_left_hand_pos:
#                     current_left_hand_pos = None
#                     if closest_center == current_chest_pos:
#                         current_chest_pos = center
#                     elif closest_center == current_right_hand_pos:
#                         current_right_hand_pos = center
#                     elif closest_center == current_left_foot_pos:
#                         current_left_foot_pos = center
#                     elif closest_center == current_right_foot_pos:
#                         current_right_foot_pos = center
#                 elif closest_limb == current_right_hand_pos:
#                     current_right_hand_pos = None
#                     if closest_center == current_chest_pos:
#                         current_chest_pos = center
#                     elif closest_center == current_left_hand_pos:
#                         current_left_hand_pos = center
#                     elif closest_center == current_left_foot_pos:
#                         current_left_foot_pos = center
#                     elif closest_center == current_right_foot_pos:
#                         current_right_foot_pos = center
#                 elif closest_limb == current_left_foot_pos:
#                     current_left_foot_pos = None
#                     if closest_center == current_chest_pos:
#                         current_chest_pos = center
#                     elif closest_center == current_left_hand_pos:
#                         current_left_hand_pos = center
#                     elif closest_center == current_right_hand_pos:
#                         current_right_hand_pos = center
#                     elif closest_center == current_right_foot_pos:
#                         current_right_foot_pos = center
#                 elif closest_limb == current_right_foot_pos:
#                     current_right_foot_pos = None
#                     if closest_center == current_chest_pos:
#                         current_chest_pos = center
#                     elif closest_center == current_left_hand_pos:
#                         current_left_hand_pos = center
#                     elif closest_center == current_right_hand_pos:
#                         current_right_hand_pos = center
#                     elif closest_center == current_left_foot_pos:
#                         current_left_foot_pos = center

#     # If there is one limb that is not the closest to any center, then allocate it to None
#     if current_chest_pos is None:
#         current_chest_pos = prev_chest_pos
#     if current_left_hand_pos is None:
#         current_left_hand_pos = prev_left_hand_pos
#     if current_right_hand_pos is None:
#         current_right_hand_pos = prev_right_hand_pos
#     if current_left_foot_pos is None:
#         current_left_foot_pos = prev_left_foot_pos
#     if current_right_foot_pos is None:
#         current_right_foot_pos = prev_right_foot_pos

#     # Add the frame to the json
#     add_frame_to_json(frame_number, current_chest_pos[0], current_chest_pos[1], current_left_hand_pos[0], current_left_hand_pos[1], current_right_hand_pos[0], current_right_hand_pos[1], current_left_foot_pos[0], current_left_foot_pos[1], current_right_foot_pos[0], current_right_foot_pos[1])

#     # Update the previous positions
#     prev_chest_pos = current_chest_pos
#     prev_left_hand_pos = current_left_hand_pos
#     prev_right_hand_pos = current_right_hand_pos
#     prev_left_foot_pos = current_left_foot_pos
#     prev_right_foot_pos = current_right_foot_pos

# # Write the body data to a file
# write_body_data()

### Attempt 2 at classification algorithm (dumb algorithm but might work in this case)

In [112]:
def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# # Assign the initial limb positions
# first_frame_centers = all_centers[0]

# # Sort the list in order of the y values
# first_frame_centers.sort(key=lambda tup: tup[1])

# initial_chest_position = first_frame_centers[len(first_frame_centers) // 2]

# initial_hand_positions = first_frame_centers[:len(first_frame_centers) // 2]

# initial_hand_positions.sort(key=lambda tup: tup[0])

# initial_left_hand_position = initial_hand_positions[0]
# initial_right_hand_position = initial_hand_positions[1]

# initial_foot_positions = first_frame_centers[len(first_frame_centers) // 2 + 1:]

# initial_foot_positions.sort(key=lambda tup: tup[0])

# initial_left_foot_position = initial_foot_positions[0]
# initial_right_foot_position = initial_foot_positions[1]

def naive_classify(centers: list[(int, int)]) -> dict[str: (int, int)]:
    locations = {
        'chest': None,
        'left-hand': None,
        'right-hand': None,
        'left-foot': None,
        'right-foot': None,
    }
    if len(centers) == 5:
        centers.sort(key=lambda tup: tup[1])
        feet_centers = centers[len(centers)//2+1:]
        chest_and_hand_centers = set(centers) - set(feet_centers)
        chest_and_hand_centers = list(chest_and_hand_centers)
        chest_and_hand_centers.sort(key=lambda tup: tup[0])
        chest_center = chest_and_hand_centers[len(chest_and_hand_centers)//2]
        chest_and_hand_centers.remove(chest_center)
        hand_centers = chest_and_hand_centers
        hand_centers.sort(key=lambda tup: tup[0])
        feet_centers.sort(key=lambda tup: tup[0])
        locations['chest'] = chest_center
        locations['left-hand'] = hand_centers[0]
        locations['right-hand'] = hand_centers[1]
        locations['left-foot'] = feet_centers[0]
        locations['right-foot'] = feet_centers[1]

        return locations
    
    return None

completed_frames = []
for frame in range(0, TOTAL_FRAMES):
    frame_locations = naive_classify(all_centers[frame])

    if frame_locations == None:
        continue

    completed_frames.append(frame)
    add_frame_to_json(frame, frame_locations['chest'][0], frame_locations['chest'][1], frame_locations['left-hand'][0], frame_locations['left-hand'][1], frame_locations['right-hand'][0], frame_locations['right-hand'][1], frame_locations['left-foot'][0], frame_locations['left-foot'][1], frame_locations['right-foot'][0], frame_locations['right-foot'][1])

write_body_data()

### Create a video with the centers labelled to ensure quality of the JSON data

In [116]:
# Open the video file for writing
path_to_output_video = './monkeyWithLabelledCenters.mp4'
path_to_output_video2 = './monkeyWithLabelledCentersBlackBackground.mp4'
out = cv2.VideoWriter(path_to_output_video, cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 30, (int(frame_width), int(frame_height)))
out2 = cv2.VideoWriter(path_to_output_video2, cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 30, (int(frame_width), int(frame_height)))

body_data = get_body_data()

# Loop through each frame and write it to the video file
for frame_number in completed_frames:
    # Load the frame
    img = cv2.imread('movie_frames/frame%d.tif' % frame_number)
    # Swap the blue and red channels
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    black_background_img = np.zeros((int(frame_height), int(frame_width), 3), np.uint8)
    
    # Get the body data for this frame
    frame_data = body_data["frames"][frame_number]

    # Draw the chest
    chest_pos = (frame_data["chest"]["x"], frame_data["chest"]["y"])
    img = cv2.circle(img, chest_pos, radius=3, color=(255, 0, 0), thickness=-1)
    img = cv2.putText(img, "Chest", (chest_pos[0] + 5, chest_pos[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
    black_background_img = cv2.circle(black_background_img, chest_pos, radius=3, color=(255, 0, 0), thickness=-1)
    black_background_img = cv2.putText(black_background_img, "Chest", (chest_pos[0] + 5, chest_pos[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

    # Draw the left hand
    left_hand_pos = (frame_data["left_hand"]["x"], frame_data["left_hand"]["y"])
    img = cv2.circle(img, left_hand_pos, radius=3, color=(0, 255, 0), thickness=-1)
    img = cv2.putText(img, "Left Hand", (left_hand_pos[0] + 5, left_hand_pos[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
    black_background_img = cv2.circle(black_background_img, left_hand_pos, radius=3, color=(0, 255, 0), thickness=-1)
    black_background_img = cv2.putText(black_background_img, "Left Hand", (left_hand_pos[0] + 5, left_hand_pos[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    # Draw the right hand
    right_hand_pos = (frame_data["right_hand"]["x"], frame_data["right_hand"]["y"])
    img = cv2.circle(img, right_hand_pos, radius=3, color=(0, 255, 0), thickness=-1)
    img = cv2.putText(img, "Right Hand", (right_hand_pos[0] + 5, right_hand_pos[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
    black_background_img = cv2.circle(black_background_img, right_hand_pos, radius=3, color=(0, 255, 0), thickness=-1)
    black_background_img = cv2.putText(black_background_img, "Right Hand", (right_hand_pos[0] + 5, right_hand_pos[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    # Draw the left foot
    left_foot_pos = (frame_data["left_foot"]["x"], frame_data["left_foot"]["y"])
    img = cv2.circle(img, left_foot_pos, radius=3, color=(0, 0, 255), thickness=-1)
    img = cv2.putText(img, "Left Foot", (left_foot_pos[0] + 5, left_foot_pos[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
    black_background_img = cv2.circle(black_background_img, left_foot_pos, radius=3, color=(0, 0, 255), thickness=-1)
    black_background_img = cv2.putText(black_background_img, "Left Foot", (left_foot_pos[0] + 5, left_foot_pos[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

    # Draw the right foot
    right_foot_pos = (frame_data["right_foot"]["x"], frame_data["right_foot"]["y"])
    img = cv2.circle(img, right_foot_pos, radius=3, color=(0, 0, 255), thickness=-1)
    img = cv2.putText(img, "Right Foot", (right_foot_pos[0] + 5, right_foot_pos[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
    black_background_img = cv2.circle(black_background_img, right_foot_pos, radius=3, color=(0, 0, 255), thickness=-1)
    black_background_img = cv2.putText(black_background_img, "Right Foot", (right_foot_pos[0] + 5, right_foot_pos[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
    
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite('movie_frames_with_centers_labelled/frame%d.tif' % frame_number, img)
    out.write(img)
    black_background_img = cv2.cvtColor(black_background_img, cv2.COLOR_RGB2BGR)
    out2.write(black_background_img)

# Release the video file
out.release()
out2.release()
cv2.destroyAllWindows()